In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

# !pip install transformers
!pip install torchxrayvision
!tar -xzf /content/gdrive/MyDrive/Colab\ Notebooks/images.tar.gz 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 9.2 MB/s eta 0:00:00
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import skimage
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms

import torchxrayvision as xrv
from tqdm.notebook import tqdm
from glob import glob
from time import time
import numpy as np

# from transformers import ViTImageProcessor, ViTModel
from PIL import Image
from tqdm.notebook import tqdm

import pandas as pd

In [4]:
feature_extractor = "densenet121-res224-chex"

import json
with open(f'features_{feature_extractor}.json') as f:
  data = json.load(f)

images = list(data.keys())
features = torch.Tensor(list(data.values()))


In [5]:
features.shape

torch.Size([2018, 1024])

Train the model

In [87]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

annotations_file = glob('/content/*image*.csv')[0]

DISEASE_NAMES = ['edema', 'consolidation', 'pleural_effusion', 'pneumothorax', 'pneumonia']
for disease_name in DISEASE_NAMES:
    print("Disease:", disease_name)
    df = pd.read_csv(annotations_file)
    df = df[~df[f'{disease_name}_progression'].isna()]
    image1 = df.previous_dicom_id.to_numpy()
    image2 = df.dicom_id.to_numpy()

    df[f'{disease_name}_progression'].replace({'worsening': 1, 'stable': 0, 'improving': 2}, inplace=True)
    
    labels = df[f'{disease_name}_progression'].to_numpy()
    
    with open(f'features_{feature_extractor}.json') as f:
      data = json.load(f)
      image_names = [d.split('/')[-1] for d in data.keys()]
      in_features = np.array(list(data.values()))
    
    features1 = np.array([in_features[image_names.index(img.split('/')[-1]+'.jpg')] for img in image1])
    features2 = np.array([in_features[image_names.index(img.split('/')[-1]+'.jpg')] for img in image2])
    features = np.concatenate([features1, features2], axis=1)
    X, X_test, y, y_test = train_test_split(features, labels, test_size=0.2, random_state=123, shuffle=True)

    model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
    model.fit(X, y)
    preds = model.predict(X_test)
    print(classification_report(y_test, preds))

    ### EXPERIMENT 2:
    print("=========== EXP2 =============================")
    feat1 = X_test[:, :X_test.shape[1]//2+1]
    feat2 = X_test[:, X_test.shape[1]//2+1:]
    inv_feat = np.concatenate([feat2, feat1], axis=1)
    preds_inv = model.predict(inv_feat)
    y_test_inv = y_test.copy()
    y_test_inv[y_test==1] = 2
    y_test_inv[y_test==2] = 1

    # print(list(zip(y_test, y_test_inv)))
    # print(list(zip(preds, preds_inv)))
    print("Number of cases where label == 0:", sum(y_test==0)) 
    print("Number of cases where label == 1:", sum(y_test==1)) 
    print("Number of cases where label == 2:", sum(y_test==2)) 
    print("Number of cases where label == 0 and output was 0:", sum( (y_test==0) & (preds_inv==0) ))
    print("Number of cases where pred == 1 and output was 2:", sum( (preds==1) & (preds_inv==2) ))
    print("Number of cases where pred == 2 and output was 1:", sum( (preds==2) & (preds_inv==1) ))
    # print(classification_report(y_test_inv, preds_inv))

Disease: edema
              precision    recall  f1-score   support

           0       0.30      0.21      0.25        14
           1       0.64      0.82      0.72        22
           2       0.38      0.33      0.35        18

    accuracy                           0.50        54
   macro avg       0.44      0.46      0.44        54
weighted avg       0.46      0.50      0.48        54

=========== EXP2 =============================
Number of cases where label == 0: 14
Number of cases where label == 1: 22
Number of cases where label == 2: 18
Number of cases where label == 0 and output was 0: 0
Number of cases where pred == 1 and output was 2: 6
Number of cases where pred == 2 and output was 1: 7
Disease: consolidation
              precision    recall  f1-score   support

           0       0.43      0.67      0.53        15
           1       0.61      0.61      0.61        18
           2       0.00      0.00      0.00         8

    accuracy                           0.51     

In [64]:
(50+51+39+37+56)/5

46.6